In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
sheet_names = ['2015','2016','2017','2018','会员等级']
sheet_datas = [pd.read_excel('D:/Code/User Value Analysis And Operation Strategy Based On Consumption Income/Data/sales.xlsx',sheet_name=i) for i in sheet_names]

In [3]:
for each_name,each_data in zip(sheet_names,sheet_datas):    
    print('[data summary for {0:=^50}]'.format(each_name))
    # 展示数据前5条
    print('Overview:','\n',each_data.head())
    # 数据描述性信息
    print('DESC:','\n',each_data.describe())
    # 统计缺失值 
    print('NA records',each_data.isnull().any(axis=1).sum())   
    # 数据类型
    print('Dtypes',each_data.dtypes) 

[data summary for =======================2015=======================]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
4  35038752292  3000821546 2015-01-01    10.1
DESC: 
                会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NA records 0
Dtypes 会员ID             int64
订单号              int64
提交日期    datetime64[ns]
订单金额           float64
dtype: object
[data summary for =======================2016======================

In [4]:
# 去除缺失值和异常值
for ind,each_data in enumerate(sheet_datas[:-1]):
    # 丢弃缺失值记录
    sheet_datas[ind] = each_data.dropna()
    # 丢弃订单金额<=1的记录
    sheet_datas[ind] = each_data[each_data['订单金额'] > 1]
    # 增加一列最大日期值
    sheet_datas[ind]['max_year_date'] = each_data['提交日期'].max() 

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
# 汇总所有数据
data_merge = pd.concat(sheet_datas[:-1],axis=0)
# 获取各自年份数据
data_merge['date_interval'] = data_merge['max_year_date']-data_merge['提交日期']
data_merge['year'] = data_merge['提交日期'].dt.year
# 转换日期间隔为数字
data_merge['date_interval'] = data_merge['date_interval'].apply(lambda x: x.days)
data_merge.head()

,会员ID,订单号,提交日期,订单金额,max_year_date,date_interval,year
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,364,2015
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,364,2015
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,364,2015
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,364,2015
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,364,2015


In [10]:
# 按会员ID做汇总
rfm_gb = data_merge.groupby(['year','会员ID'],as_index=False).agg({'date_interval': 'min',  # 计算最近一次订单时间
                                                   '提交日期': 'count', # 计算订单频率
                                                   '订单金额': 'sum'})  # 计算订单总金额
# 重命名列名
rfm_gb.columns =  ['year','会员ID','r','f','m']
rfm_gb.head()

,year,会员ID,r,f,m
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0


In [11]:
# 查看数据分布
desc_pd = rfm_gb.iloc[:,2:].describe().T
print(desc_pd)
# 定义区间边界
r_bins = [-1,79,255,365] # 注意起始边界小于最小值
f_bins = [0,2,5,130] 
m_bins = [0,69,1199,206252]

      count         mean          std  min   25%    50%     75%       max
r  148591.0   165.524043   101.988472  0.0  79.0  156.0   255.0     365.0
f  148591.0     1.365002     2.626953  1.0   1.0    1.0     1.0     130.0
m  148591.0  1323.741329  3753.906883  1.5  69.0  189.0  1199.0  206251.8


In [13]:
# RFM分箱得分
# 计算R得分
rfm_gb['r_score'] = pd.cut(rfm_gb['r'], r_bins, labels=[i for i in range(len(r_bins)-1,0,-1)])
# 计算F得分
rfm_gb['f_score'] = pd.cut(rfm_gb['f'], f_bins, labels=[i+1 for i in range(len(f_bins)-1)])
# 计算M得分
rfm_gb['m_score'] = pd.cut(rfm_gb['m'], m_bins, labels=[i+1 for i in range(len(m_bins)-1)])  

In [14]:
print(rfm_gb.head())

   year  会员ID    r  f       m r_score f_score m_score
0  2015   267  197  2   105.0       2       1       2
1  2015   282  251  1    29.7       2       1       1
2  2015   283  340  1  5398.0       1       1       3
3  2015   343  300  1   118.0       1       1       2
4  2015   525   37  3   213.0       3       2       2


In [15]:
# 方法一：RFM组合
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] = rfm_gb['m_score'].astype(np.str)
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(rfm_gb['m_score'])

In [16]:
print(rfm_gb.head())

   year  会员ID    r  f       m r_score f_score m_score rfm_group
0  2015   267  197  2   105.0       2       1       2       212
1  2015   282  251  1    29.7       2       1       1       211
2  2015   283  340  1  5398.0       1       1       3       113
3  2015   343  300  1   118.0       1       1       2       112
4  2015   525   37  3   213.0       3       2       2       322


In [17]:
# 匹配会员等级和rfm得分
rfm_merge = pd.merge(rfm_gb,sheet_datas[-1],on='会员ID',how='inner')

In [18]:
# 随机森林获得rfm因子得分
clf = RandomForestClassifier()
clf = clf.fit(rfm_merge[['r','f','m']],rfm_merge['会员等级'])
weights = clf.feature_importances_
print('feature importance:',weights)

feature importance: [0.40650885 0.00584077 0.58765038]


In [19]:
# 方法二：加权得分
rfm_gb = rfm_gb.apply(np.int32) # cate转数值
rfm_gb['rfm_score'] = rfm_gb['r_score'] * weights[0] + rfm_gb['f_score'] * weights[1] + rfm_gb['m_score'] * weights[2]

In [20]:
print(rfm_gb.head())

   year  会员ID    r  f     m  r_score  f_score  m_score  rfm_group  rfm_score
0  2015   267  197  2   105        2        1        2        212   1.994159
1  2015   282  251  1    29        2        1        1        211   1.406509
2  2015   283  340  1  5398        1        1        3        113   2.175301
3  2015   343  300  1   118        1        1        2        112   1.587650
4  2015   525   37  3   213        3        2        2        322   2.406509


In [21]:
rfm_gb.to_excel('D:/Code/User Value Analysis And Operation Strategy Based On Consumption Income/Result/sales_rfm_score.xlsx')